In [17]:
#Importing libraries and dependencies
import pandas as pd
import requests
import numpy as np
import hvplot.pandas
import matplotlib.pyplot as plt
import datetime as dt
import json
import csv
#I dont need all of these, need to comeback 
# and just scrap the ones i dont need later

In [28]:
#Setting api headpoint, and headers to accept extra info
url = "https://api.quiverquant.com/beta/bulk/congresstrading"
headers = {
    'Accept': "application/json",
    #Will probably need to change this to have api held in a .env
    'Authorization': "Bearer"
}
#Getting the data
response = requests.get(url, headers=headers)
#Checking if the data was returned
if response.status_code == 200:
    print('200')
    #load json response into the dataframe
    df = pd.read_json(response.content.decode("utf-8"))
    print(df.head())
    print(df.dtypes)
else :
    print(response.status_code)

500


In [29]:
#Convert the TransactionDate to  datetime
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
print(df.dtypes)

Representative             object
BioGuideID                 object
TransactionDate    datetime64[ns]
Ticker                     object
Transaction                object
Range                      object
House                      object
Amount                    float64
Party                      object
dtype: object


In [30]:
#Check how far back the data goes.
def checkOldest() :
    oldest_date = df['TransactionDate'].min()
    print(oldest_date)
checkOldest()

2019-01-04 00:00:00


In [31]:
# Filter out any data from before 2018 winners took office
inauguration_date =  pd.to_datetime('2019-01-03')
df = df.loc[df['TransactionDate']> inauguration_date]
#Verify old data was removed
checkOldest()
# print(df.head())

2019-01-04 00:00:00


In [32]:
#Drop columns I don't want
df = df.drop(columns=['ReportDate', 'last_modified'])
# Checking for ugly dirty data 
print(df.isnull().sum())
print(df.duplicated().sum())

KeyError: "['ReportDate', 'last_modified'] not found in axis"

In [ ]:
#Dropping duplicate rows
# df = df.drop_duplicates(keep='first')
#I am scared that this will mess with the data,
#Like if the same person did the same purchase twice

In [ ]:
#Since plotting will be handled in a diffierent file

#We can split data sets here into different dfs for analysis in 
# other files in a dictionary

#I don't really know which dataframes i will need later 
# so i am just making a bunch

In [ ]:
#Dictionary with id as key and name as value
#Later we can track most active ids, 
# and find most active names as a result
rep_to_id_dict = df[['Representative', 'BioGuideID']].set_index('Representative')['BioGuideID'].to_dict()
df = df.reset_index(drop=True)

#Turns out jupyter hates sharing dicitonaries across files
#Doing it through json
with open('rep_to_id.json', 'w') as f:
    json.dump(rep_to_id_dict, f)
    
#Making sure it worked
print(rep_to_id_dict)

#I'm pretty sure I can just drop representative name now
#When i need to check names, i can compare it against this dict
df.drop(columns=['Representative'])

#Remember to create a function that checks representative name
#In my head, this makes df smaller and therefor easier to  work with

{'Rudy Yakym Iii': 'Y000067', 'Marjorie Taylor Mrs Greene': 'G000596', 'Pete Sessions': 'S000250', 'William R. Keating': 'K000375', 'Max Miller': 'M001222', 'Laurel Lee': 'L000597', 'John James': 'J000307', 'Michael C. Burgess': 'B001248', 'A. Mitchell Jr. McConnell': 'M000355', 'Suzan K. Delbene': 'D000617', 'Josh Gottheimer': 'G000583', 'John Hickenlooper': 'H000273', 'Mark Dr Green': 'G000590', 'Shelley Moore Capito': 'C001047', 'Eleanor Holmes Norton': 'N000147', 'Tommy Tuberville': 'T000278', 'Jonathan Jackson': 'J000309', 'Greg Landsman': 'L000601', 'Markwayne Mullin': 'M001190', 'Virginia Foxx': 'F000450', 'Morgan Mcgarvey': 'M001220', 'Jake Auchincloss': 'A000148', 'Sheldon Whitehouse': 'W000802', 'Lloyd Doggett': 'D000399', 'Earl Blumenauer': 'B000574', 'C. Scott Franklin': 'F000472', 'Richard W. Allen': 'A000372', 'Carol Devine Miller': 'M001205', 'Kathy Manning': 'M001135', 'Debbie Wasserman Schultz': 'W000797', 'James E Hon Banks': 'B001299', 'Susan M. Collins': 'C001035', 

,BioGuideID,TransactionDate,Ticker,Transaction,Range,House,Amount,Party
0,Y000067,2024-09-26,MATURE,Purchase,"$1,001 - $15,000",Representatives,1001.0,R
1,G000596,2024-09-20,GOOG,Purchase,"$1,001 - $15,000",Representatives,1001.0,R
2,G000596,2024-09-20,CRWD,Purchase,"$1,001 - $15,000",Representatives,1001.0,R
3,G000596,2024-09-20,BX,Purchase,"$1,001 - $15,000",Representatives,1001.0,R
4,G000596,2024-09-20,LRCX,Purchase,"$1,001 - $15,000",Representatives,1001.0,R
...,...,...,...,...,...,...,...,...
27846,P000616,2019-01-04,WFC,Sale,"$15,001 - $50,000",Representatives,15001.0,D
27847,T000479,2019-01-04,PYPL,Purchase,"$1,001 - $15,000",Representatives,1001.0,R
27848,P000616,2019-01-04,BLK,Purchase,"$1,001 - $15,000",Representatives,1001.0,D
27849,P000616,2019-01-04,M,Purchase,"$1,001 - $15,000",Representatives,1001.0,D


In [ ]:
#Now that I started this I don't think this part is necesarry...
#But i am keeping it here if i later realize that it is necesarry...

In [ ]:
#In the meantime... export that goodness to a csv
df.to_csv('bulk_output.csv', index=False)